In [ ]:
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
!unzip /content/Dataset.zip

Archive:  /content/Dataset.zip
   creating: Dataset/EarlyBlight/
  inflating: Dataset/EarlyBlight/EarlyBlight_1.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_10.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_100.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1000.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1001.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1002.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1003.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1004.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1005.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1006.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1007.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1008.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1009.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_101.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1010.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1011.jpg  
  inflating: Dataset/EarlyBlight/EarlyBlight_1

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
train_batch = train_gen.flow_from_directory(
    directory="/content/Dataset",
    target_size=(300, 300),
    batch_size=64,
    subset='training',
    class_mode = 'categorical',
    seed=64
)
valid_batch = train_gen.flow_from_directory(
    directory="/content/Dataset",
    target_size=(300, 300),
    batch_size=64,
    subset='validation',
    class_mode = 'categorical',
    seed=64
)

Found 3259 images belonging to 3 classes.
Found 813 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

80134624/80134624 [==============================] - 2s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
output = Dense(3, activation='softmax')(x)  # Replace 'num_classes' with your actual number of classes

In [ ]:
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_batch,
                    steps_per_epoch=len(train_batch),
                    epochs=20,
                    validation_data=valid_batch,
                    validation_steps=len(valid_batch))

Epoch 1/20
51/51 [==============================] - 137s 2s/step - loss: 1.1687 - accuracy: 0.6563 - val_loss: 0.8760 - val_accuracy: 0.5916
Epoch 2/20
51/51 [==============================] - 96s 2s/step - loss: 0.3884 - accuracy: 0.8460 - val_loss: 0.5634 - val_accuracy: 0.7626
Epoch 3/20
51/51 [==============================] - 97s 2s/step - loss: 0.3373 - accuracy: 0.8678 - val_loss: 0.7003 - val_accuracy: 0.7036
Epoch 4/20
51/51 [==============================] - 96s 2s/step - loss: 0.2695 - accuracy: 0.9006 - val_loss: 0.6073 - val_accuracy: 0.7651
Epoch 5/20
51/51 [==============================] - 95s 2s/step - loss: 0.2752 - accuracy: 0.8978 - val_loss: 0.7050 - val_accuracy: 0.7491
Epoch 6/20
51/51 [==============================] - 96s 2s/step - loss: 0.2926 - accuracy: 0.8972 - val_loss: 0.7410 - val_accuracy: 0.6974
Epoch 7/20
51/51 [==============================] - 98s 2s/step - loss: 0.3115 - accuracy: 0.8819 - val_loss: 1.1235 - val_accuracy: 0.6384
Epoch 8/20
51/51 [=

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image
img = image.load_img('/content/Dataset/EarlyBlight/EarlyBlight_1.jpg', target_size=(300, 300))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension

# Make predictions
predictions = model.predict(img_array)  # Assuming 'model' is your trained VGG19 model

# Get the class with the highest probability
predicted_class_index = np.argmax(predictions)

# Define your class labels
class_labels = ['class1', 'class2', 'class3']  # Update with your actual class labels

# Get the predicted class label
predicted_class = class_labels[predicted_class_index]

print("Predicted class:", predicted_class)

1/1 [==============================] - 1s 840ms/step
Predicted class: class1


In [ ]:
import pickle

# Save the trained model to a file using pickle
with open('trained_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)